In [31]:
import pandas as pd
from pathlib import Path
import os
import csv
from pymongo import MongoClient

In [32]:
shark_attackDF = pd.read_csv("Desktop/Shark_Attacks.csv")
shark_attackDF.head()

/var/folders/fb/7f3ntl4s41z8z9s2_hfk1b1w0000gn/T/ipykernel_98395/2672375721.py:1: DtypeWarning: Columns (0,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  shark_attackDF = pd.read_csv("Desktop/Shark_Attacks.csv")


,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,...,Unnamed: 245,Unnamed: 246,Unnamed: 247,Unnamed: 248,Unnamed: 249,Unnamed: 250,Unnamed: 251,Unnamed: 252,Unnamed: 253,Unnamed: 254
0,15 Mar 2024,2024.0,Unprovoked,AUSTRALIA,Queensland,Bargara Beach,Swimming,Brooklyn Sauer,F,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,04 Mar 2024,2024.0,Unprovoked,USA,Hawaii,"Old Man's, Waikiki",Surfing,Matthew White,M,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,02 Mar-2024,2024.0,Unprovoked,USA,Hawaii,"Rainbows, Oahu",Swimming,NaN,F,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,25 Feb-2024,2024.0,Unprovoked,AUSTRALIA,Western Australia,"Sandlnd Island, Jurian Bay",NaN,female,F,46,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14 Feb-2024,2024.0,Unprovoked,INDIA,Maharashtra,"Vaitarna River, Palghar District",Fishing,Vicky Suresh Govari,M,32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
# Remove unnecesary columns
columns = ["Year", "Type", "Country", "State", "Activity", 
           "Sex", "Age", "Injury"]
shark_attackDF = shark_attackDF[columns]
shark_attackDF.reset_index(inplace=True)
shark_attackDF.rename(columns={"index": "id"}, inplace=True)
shark_attackDF.head()

,id,Year,Type,Country,State,Activity,Sex,Age,Injury
0,0,2024.0,Unprovoked,AUSTRALIA,Queensland,Swimming,F,13,"Minor injuries to back, abdomen and legs"
1,1,2024.0,Unprovoked,USA,Hawaii,Surfing,M,NaN,"No injury, shark bit surfboard"
2,2,2024.0,Unprovoked,USA,Hawaii,Swimming,F,11,Lacerations to left foot
3,3,2024.0,Unprovoked,AUSTRALIA,Western Australia,NaN,F,46,Leg bitten
4,4,2024.0,Unprovoked,INDIA,Maharashtra,Fishing,M,32,Calf of lower left leg injured


In [34]:
shark_attackDF.dtypes

id            int64
Year        float64
Type         object
Country      object
State        object
Activity     object
Sex          object
Age          object
Injury       object
dtype: object

In [35]:
# take period out of year column
shark_attackDF['Year'] = shark_attackDF['Year'].round().astype('Int64')

In [36]:
shark_attackDF.head()

,id,Year,Type,Country,State,Activity,Sex,Age,Injury
0,0,2024,Unprovoked,AUSTRALIA,Queensland,Swimming,F,13,"Minor injuries to back, abdomen and legs"
1,1,2024,Unprovoked,USA,Hawaii,Surfing,M,NaN,"No injury, shark bit surfboard"
2,2,2024,Unprovoked,USA,Hawaii,Swimming,F,11,Lacerations to left foot
3,3,2024,Unprovoked,AUSTRALIA,Western Australia,NaN,F,46,Leg bitten
4,4,2024,Unprovoked,INDIA,Maharashtra,Fishing,M,32,Calf of lower left leg injured


In [37]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [38]:
# confirm that our new database was created
print(mongo.list_database_names())

['ETLDemoDB01', 'admin', 'autosaurus', 'classDB', 'config', 'fruits_db', 'local', 'met', 'shark_attacks', 'uk_food']


In [39]:
# assign the database to a variable name
db = mongo.shark_attacks

In [40]:
# review the collections in our new database
print(db.list_collection_names())

['incidents']


In [41]:
shark_attackDF_finalset = shark_attackDF.loc[(shark_attackDF['Year'] >= 2004) & (shark_attackDF['Year'] <= 2024)]
shark_attackDF_finalset.tail()

,id,Year,Type,Country,State,Activity,Sex,Age,Injury
2320,2320,2004,Unprovoked,AUSTRALIA,New South Wales,Diving,M,NaN,"No injury, swim fin ripped off"
2321,2321,2004,Unprovoked,SOUTH AFRICA,Eastern Cape Province,Surfing,M,22,Foot lacerated
2322,2322,2004,Invalid,NEW ZEALAND,North Island,Kayaking (returning from spearfishing),M,NaN,"No injury, no attack, shark took fish from bac..."
2323,2323,2004,Sea Disaster,EGYPT,Sinai Peninsula,Air disaster. Flash Airlines Boeing 737 crashe...,NaN,NaN,"No survivors, sharks scavenged on remains"
2324,2324,2004,Unprovoked,AUSTRALIA,Western Australia,Surfing,M,NaN,No inury


In [42]:
# convert data into dictionary
#shark_attackDF_finalset.reset_index(inplace=True)
shark_attackDF_finalset_dict = shark_attackDF_finalset.to_dict()

In [43]:
from pprint import pprint
pprint(shark_attackDF_finalset_dict)

{'Activity': {0: 'Swimming',
              1: 'Surfing',
              2: 'Swimming',
              3: nan,
              4: 'Fishing',
              5: 'Spearfishing',
              6: 'Swimming',
              7: 'Swimming',
              8: 'Surfing',
              9: 'Fishing',
              10: 'Surfing',
              11: 'Free diving',
              12: 'Surfing',
              13: 'Surfing',
              14: 'Foil Boarding',
              15: 'Wading',
              16: 'Swimming',
              17: 'Swimming',
              18: 'Snorkeling',
              19: 'Paddle boarding',
              20: 'Swimming',
              21: 'Swimming',
              22: 'Scuba diving',
              23: 'Free diving',
              24: 'Swimming',
              25: 'Spearfishing',
              26: 'Free diving',
              27: 'Scuba diving',
              28: 'Surfing',
              29: 'Surfing',
              30: 'Fishing',
              31: 'Fishing',
              32: 'Snorkeling',

In [45]:
# export dataframe to mongoDB
#db = ['shark_attacks']
collection = db['incidents']
shark_attackDF_finalset.reset_index(inplace=True)
shark_attackDF_finalset_dict = shark_attackDF_finalset.to_dict("records")

# Insert collection
collection.insert_many(shark_attackDF_finalset_dict)

InsertManyResult([ObjectId('6696ff4dc10ca7acadf71537'), ObjectId('6696ff4dc10ca7acadf71538'), ObjectId('6696ff4dc10ca7acadf71539'), ObjectId('6696ff4dc10ca7acadf7153a'), ObjectId('6696ff4dc10ca7acadf7153b'), ObjectId('6696ff4dc10ca7acadf7153c'), ObjectId('6696ff4dc10ca7acadf7153d'), ObjectId('6696ff4dc10ca7acadf7153e'), ObjectId('6696ff4dc10ca7acadf7153f'), ObjectId('6696ff4dc10ca7acadf71540'), ObjectId('6696ff4dc10ca7acadf71541'), ObjectId('6696ff4dc10ca7acadf71542'), ObjectId('6696ff4dc10ca7acadf71543'), ObjectId('6696ff4dc10ca7acadf71544'), ObjectId('6696ff4dc10ca7acadf71545'), ObjectId('6696ff4dc10ca7acadf71546'), ObjectId('6696ff4dc10ca7acadf71547'), ObjectId('6696ff4dc10ca7acadf71548'), ObjectId('6696ff4dc10ca7acadf71549'), ObjectId('6696ff4dc10ca7acadf7154a'), ObjectId('6696ff4dc10ca7acadf7154b'), ObjectId('6696ff4dc10ca7acadf7154c'), ObjectId('6696ff4dc10ca7acadf7154d'), ObjectId('6696ff4dc10ca7acadf7154e'), ObjectId('6696ff4dc10ca7acadf7154f'), ObjectId('6696ff4dc10ca7acadf715